

# Grounding with Bing
**Grounding with Bing Search** allows your Azure AI Agents to incorporate real-time public web data when generating responses. You need to create a Grounding with Bing Search resource, and then connect this resource to your Azure AI Agents. When a user sends a query, Azure AI Agents decide if Grounding with Bing Search should be leveraged or not. If so, it will leverage Bing to search over public web data and return relevant chunks. Lastly, Azure AI Agents will use returned chunks to generate a response.  

You can ask questions such as "*what is the top news today*" or "*what is the recent update in the retail industry in the US?*", which require real-time public data.

Developers and end users don't have access to raw content returned from Grounding with Bing Search. The response, however, includes citations with links to the websites used to generate the response, and a link to the Bing query used for the search. These two *References* must be retained and displayed in the exact form provided by Microsoft, as per Grounding with Bing Search's [Use and Display Requirements](https://www.microsoft.com/en-us/bing/apis/grounding-legal#use-and-display-requirements). See the [how to display Grounding with Bing Search results](#how-to-display-grounding-with-bing-search-results) section for details.

>[!IMPORTANT]
> 1. Your usage of Grounding with Bing Search may incur costs. See the [pricing page](https://www.microsoft.com/bing/apis/grounding-pricing) for details.
> 1. By creating and using a Grounding with Bing Search resource through code-first experience, such as Azure CLI, or deploying through deployment template, you agree to be bound by and comply with the terms available at https://www.microsoft.com/en-us/bing/apis/grounding-legal, which may be updated from time to time.


## **Setup:**  

> [!NOTE]
> 1. Grounding with Bing Search only works with the following Azure OpenAI models: `gpt-3.5-turbo-0125`, `gpt-4-0125-preview`, `gpt-4-turbo-2024-04-09`, `gpt-4o-0513`

1. Register the Bing Search provider.

   ```console
       az provider register --namespace 'Microsoft.Bing'
   ```

1. Create a new Grounding with Bing Search resource in the [Azure portal](https://portal.azure.com/#create/Microsoft.BingGroundingSearch), and select the different fields in the creation form. Make sure you create this Grounding with Bing Search resource in the same resource group as your Azure AI Agent, AI Project, and other resources.

1. After you have created a Grounding with Bing Search resource, you can find it in [Azure portal](https://portal.azure.com/#home). Navigate to the resource group you've created the resource in, search for the Grounding with Bing Search resource you have created.

   ![alt text](assets/img/resource-azure-portal.png)

1. Select the Grounding with Bing Search resource you have created and copy any of the API keys.

    ![alt text](assets/img/key-resource-azure-portal.png)

1. Go to the [Azure AI Foundry portal](https://ai.azure.com/) and select the AI Project (make sure it's in the same resource group of your Grounding with Bing Search resource). Click **management center**.

    ![alt text](assets/img/project-settings-button.png)

1. Select **+ new connection** in the settings page. 

    >[!NOTE]
    > If you re-generate the API key at a later date, you need to update the connection with the new key.

    ![alt text](assets/img/project-connections.png)

1. Select **API key** in **other resource types**.

    ![alt text](assets/img/api-key-connection.png)

1. Enter the following information and then create a new connection to your Grounding with Bing Search resource.

    - Endpoint: `https://api.bing.microsoft.com/`
    - Key: `YOUR_API_KEY`
    - Connection name: `YOUR_CONNECTION_NAME` (You will use this connection name in the sample code below.)
    - Access: you can choose either *this project only* or *shared to all projects*. Just make sure in the sample code below, the project you entered connection string for has access to this connection.

Now that we have configured everything, let's create a client object, which will contain the connection string for connecting to your AI project and other resources. To make the Grounding with Bing search tool available to your agent, use a connection to initialize the tool and attach it to the agent. You can find your connection in the **connected resources** section of your project in the Azure AI Foundry portal.



In [1]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import BingGroundingTool


# Create an Azure AI Client from a connection string, copied from your Azure AI Foundry project.
# At the moment, it should be in the format "<HostName>;<AzureSubscriptionId>;<ResourceGroup>;<HubName>"
# Customer needs to login to Azure subscription via Azure CLI and set the environment variables

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

In [2]:
bing_connection = project_client.connections.get(
    connection_name=os.environ["BING_CONNECTION_NAME"]
)
conn_id = bing_connection.id

print(conn_id)

# Initialize agent bing tool and add the connection id
bing = BingGroundingTool(connection_id=conn_id)

# Create agent with the bing tool and process assistant run
project_client.__enter__()
agent = project_client.agents.create_agent(
    model="gpt-4-turbo",
    name="my-assistant",
    instructions="You are a helpful assistant",
    tools=bing.definitions,
    headers={"x-ms-enable-preview": "true"}
)
print(f"Created agent, ID: {agent.id}")

/subscriptions/3490bb6d-9e8c-46c5-b635-f8fea8df3d69/resourceGroups/agent-quickstart-rg2/providers/Microsoft.MachineLearningServices/workspaces/project-demo-agents-oaic/connections/bing_grounding
Created agent, ID: asst_9nMPjXiesA2UFWP8rPGjmF3F


In [3]:
# Create thread for communication
thread = project_client.agents.create_thread()
print(f"Created thread, ID: {thread.id}")

# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="What is the top news today",
)
print(f"Created message, ID: {message.id}")

Created thread, ID: thread_nvz9nRB6xt0WMoiFMdvexjaD
Created message, ID: msg_Z08PxBv6qPm3qWNfmY9iKoOd


In [4]:
# Create and process agent run in thread with tools
run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
print(f"Run finished with status: {run.status}")
print(f"Run ID: {run.id}")

# Retrieve run step details to get Bing Search query link
# To render the webpage, we recommend you replace the endpoint of Bing search query URLs with `www.bing.com` and your Bing search query URL would look like "https://www.bing.com/search?q={search query}"
run_steps = project_client.agents.list_run_steps(run_id=run.id, thread_id=thread.id)
run_steps_data = run_steps['data']
print(f"Last run step detail: {run_steps_data}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")


messages = project_client.agents.list_messages(thread_id=thread.id)
print(f"Messages: {messages}")

Run finished with status: completed
Run ID: run_SUwSsszYEK4EnTiLLuIxdJ0Q
Last run step detail: [{'id': 'step_oYnFq52ZF1XWQ7IZAuRMdaRQ', 'object': 'thread.run.step', 'created_at': 1736934865, 'run_id': 'run_SUwSsszYEK4EnTiLLuIxdJ0Q', 'assistant_id': 'asst_9nMPjXiesA2UFWP8rPGjmF3F', 'thread_id': 'thread_nvz9nRB6xt0WMoiFMdvexjaD', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1736934870, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type': 'message_creation', 'message_creation': {'message_id': 'msg_WGds8HjJwNFgbNPae8IIWKtI'}}, 'usage': {'prompt_tokens': 1659, 'completion_tokens': 197, 'total_tokens': 1856}}, {'id': 'step_9spIN6KL3FVXS3042aLzkFHF', 'object': 'thread.run.step', 'created_at': 1736934864, 'run_id': 'run_SUwSsszYEK4EnTiLLuIxdJ0Q', 'assistant_id': 'asst_9nMPjXiesA2UFWP8rPGjmF3F', 'thread_id': 'thread_nvz9nRB6xt0WMoiFMdvexjaD', 'type': 'tool_calls', 'status': 'completed', 'cancelled_at': None, 'completed_a

In [5]:
# Delete the agent once done
project_client.agents.delete_agent(agent.id)
print("Deleted agent")
project_client.__exit__(None, None, None)

Deleted agent


> **Important Considerations:** 
>    * According to Grounding with Bing's [terms of use and use and display requirements](https://www.microsoft.com/en-us/bing/apis/grounding-legal#use-and-display-requirements), you need to display both website URLs and Bing search query URLs in your custom interface. You can find website URLs through `annotations` parameter in API response and Bing search query URLs through `runstep` details. To render the webpage, we recommend you replace the endpoint of Bing search query URLs with `www.bing.com` and your Bing search query URL would look like "https://www.bing.com/search?q={search query}"
>    * Microsoft will use data you send to Grounding with Bing to improve Microsoft products and services. Where you send personal data to this service, you are responsible for obtaining sufficient consent from the data subjects. The Data Protection Terms in the Online Services Terms do not apply to Grounding with Bing. 

### Grounding with AI Search

### Grounding with file to assistant API